In [2]:
import os
import pandas as pd
import numpy as np
import warnings
from xgboost import XGBClassifier

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from psod.outlier_detection.psod import PSOD
from psod.preprocessing.full_preprocessing import auto_preprocess

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

SEED = 42 # Muito importante manter a SEED igual em todos os modelos para garantir a consistência dos dados no ensemble
FOLDS = 5 # Muito importante manter o mesmo número de FOLDS em todos os modelos para garantir a consistência dos dados no ensemble

/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
sintetico = pd.read_csv('../../src/train/train.csv', index_col='id')
original = pd.read_csv('../../src/train/original.csv')
test = pd.read_csv('../../src/test/test.csv', index_col='id')

train = pd.concat([sintetico, original], ignore_index=True)

initial_features = list(test.columns)

In [14]:
def cross_validation(model, X, y, encoder, stdevs_to_outlier, scoring=accuracy_score):

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

    scores = []
    out_of_fold = []
    
    for i, (train_index, val_index) in enumerate(skf.split(X, y)):
        
        print(f"Fold {i + 1}")
        
        X_train = X.iloc[train_index]
        y_train = y.iloc[train_index]

        X_val = X.iloc[val_index]
        y_val = y.iloc[val_index]

        iso_class = PSOD(stdevs_to_outlier=stdevs_to_outlier, flag_outlier_on="both ends")
        psod_outlier_class = iso_class.fit_predict(auto_preprocess(X_train), return_class=True)

        X_train["psod_outliers"] = psod_outlier_class
        X_train = X_train[X_train["psod_outliers"] == 0]
        X_train = X_train.drop("psod_outliers", axis=1)
        X_train = X_train.reset_index(drop=True)

        # Filtrando os outliers em y_train
        y_train = y_train.iloc[X_train.index]
        
        model.fit(X_train, y_train)

        probabilidades = model.predict_proba(X_val)

        # Recuperar a predição final a partir das probabilidades
        indices_predicoes = np.argmax(probabilidades, axis=1)
        classes_preditas = model.classes_[indices_predicoes]

        score = scoring(y_val, classes_preditas)

        scores.append(score)

        true_label = pd.Series(y_val.values, name='true')

        pred_label_df = pd.DataFrame(probabilidades)

        oof_pred = pd.concat([pred_label_df, true_label], axis=1, ignore_index=True)
        oof_pred.columns = [f'pred_{encoder.classes_[model.classes_[0]]}', f'pred_{encoder.classes_[model.classes_[1]]}', f'pred_{encoder.classes_[model.classes_[2]]}', 'true']

        out_of_fold.append(oof_pred)

    print(f"Score: {np.mean(scores)}")
    
    return scores, out_of_fold

In [5]:
def save_oof(oof):

    os.makedirs('oof', exist_ok=True)

    for i, fold in enumerate(oof):
        fold.to_csv(f'oof/fold_{i+1}.csv', index=False)

In [27]:
def predict_test(model, X_train, y_train, X_test, std_threshold, encoder):

    iso_class = PSOD(stdevs_to_outlier=std_threshold, flag_outlier_on="both ends")
    psod_outlier_class = iso_class.fit_predict(auto_preprocess(X_train), return_class=True)

    X_train["psod_outliers"] = psod_outlier_class
    X_train = X_train[X_train["psod_outliers"] == 0]
    X_train = X_train.drop("psod_outliers", axis=1)
    X_train = X_train.reset_index(drop=True)

    # Filtrando os outliers em y_train
    y_train = y_train.iloc[X_train.index]

    model.fit(X_train, y_train)

    probabilidades = model.predict_proba(X_test)
    pred_label_df = pd.DataFrame(probabilidades)

    pred_label_df.columns = [f'pred_{encoder[model.classes_[0]]}', f'pred_{encoder[model.classes_[1]]}', f'pred_{encoder[model.classes_[2]]}']

    os.makedirs('test', exist_ok=True)

    pred_label_df.to_csv(f'test/test_pred.csv', index=False)

    return pred_label_df

In [51]:
le = LabelEncoder()

y = pd.DataFrame(le.fit_transform(train['Target']), columns=['Target'])
y = y['Target']
X = train[initial_features]

In [83]:
# Função objetivo para a otimização
def objective(trial: Trial, X_cp, y):

    X = X_cp.copy()

    param = {
        'verbosity': 0,
        'objective': 'multi:softmax',
        'num_class': 3,
        'eval_metric': 'mlogloss',
        'booster': 'gbtree',
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'eta': trial.suggest_loguniform('eta', 1e-3, 0.3),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-8, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.5, 1.0),
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 1e-8, 1.0),
        'enable_categorical': True,
    }

    threshold = 25
    std_threshold = 10

    num_features = []
    for col in initial_features:
        unique_values = X[col].nunique()
        
        if unique_values <= threshold:
            X[col] = X[col].astype('category').cat.as_ordered()
        else:
            num_features.append(col)

    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
    ])

    xgb = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(**param)),
    ])
    
    scores,_ = cross_validation(xgb, X, y, le, std_threshold)
    accuracy = np.mean(scores)


    return accuracy

In [84]:
warnings.filterwarnings("ignore")

# Configuração do estudo
study = optuna.create_study(direction='maximize', sampler=TPESampler(), pruner=MedianPruner())
study.optimize(lambda trial: objective(trial, X, y), n_trials=2000)

[I 2024-06-30 15:25:20,352] A new study created in memory with name: no-name-bb4882dc-22eb-4f49-8c46-61c67e8501fb


Fold 1
Memory usage of dataframe is 7.48 MB
Memory usage after optimization is: 4.63 MB
Decreased by 38.1%


100%|██████████| 36/36 [00:10<00:00,  3.55it/s]


Fold 2
Memory usage of dataframe is 7.48 MB
Memory usage after optimization is: 4.63 MB
Decreased by 38.1%


100%|██████████| 36/36 [00:10<00:00,  3.59it/s]


...

Fold 5
Memory usage of dataframe is 7.48 MB
Memory usage after optimization is: 4.63 MB
Decreased by 38.1%


100%|██████████| 36/36 [00:08<00:00,  4.25it/s]
[I 2024-06-30 18:13:57,188] Trial 237 finished with value: 0.770613459281938 and parameters: {'lambda': 1.4859630091488536e-08, 'alpha': 0.005722533094930668, 'eta': 0.08004427797829111, 'gamma': 1.8944260332103266e-07, 'max_depth': 9, 'min_child_weight': 0.00010124844984913917, 'subsample': 0.8808802921960032, 'colsample_bytree': 0.5587624878221673, 'colsample_bylevel': 0.9489234291321694, 'colsample_bynode': 0.7080224129443947, 'scale_pos_weight': 1.1010402537361044e-06}. Best is trial 160 with value: 0.7716389277859876.


Score: 0.770613459281938
Fold 1
Memory usage of dataframe is 7.48 MB
Memory usage after optimization is: 4.63 MB
Decreased by 38.1%


 83%|████████▎ | 30/36 [00:06<00:01,  4.46it/s]
[W 2024-06-30 18:14:04,191] Trial 238 failed with parameters: {'lambda': 1.490401725714851e-08, 'alpha': 0.39743016098963185, 'eta': 0.09257347515476255, 'gamma': 2.3532382065675833e-07, 'max_depth': 9, 'min_child_weight': 0.0002221583430299194, 'subsample': 0.8438776970073313, 'colsample_bytree': 0.5468430618149963, 'colsample_bylevel': 0.9221284561556777, 'colsample_bynode': 0.7193996956856874, 'scale_pos_weight': 8.726843490689613e-07} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/rafael/anaconda3/envs/penidoEnv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_293936/3520595897.py", line 5, in <lambda>
    study.optimize(lambda trial: objective(trial, X, y), n_trials=2000)
  File "/tmp/ipykernel_293936/3518528801.py", line 48, in objective
    scores,_ = cross_validation(xgb, X, y, le, std_thre

KeyboardInterrupt: 

In [85]:
# Imprimir os melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros: ", study.best_params)
print("Melhor acurácia: ", study.best_value)

Melhores hiperparâmetros:  {'lambda': 3.734886738366808e-08, 'alpha': 0.23449226497585715, 'eta': 0.06543016095206454, 'gamma': 9.205817056121733e-08, 'max_depth': 9, 'min_child_weight': 0.004503075789285198, 'subsample': 0.8445748709335497, 'colsample_bytree': 0.5705190401636037, 'colsample_bylevel': 0.9807413667521441, 'colsample_bynode': 0.6899591837306497, 'scale_pos_weight': 1.0106896490147942e-08}
Melhor acurácia:  0.7716389277859876


In [21]:
xgb_params = {'lambda': 0.8420479533629325, 'alpha': 0.3032570019463072, 'eta': 0.12070324731278194, 'gamma': 2.8166311484864326e-06, 'max_depth': 6, 'min_child_weight': 0.01548641279823428, 'subsample': 0.7579361032269639, 'colsample_bytree': 0.7075974355298302, 'colsample_bylevel': 0.7355438749193131, 'colsample_bynode': 0.7901425486028625, 'scale_pos_weight': 1.1929478083989252e-05}

threshold = 1
std_threshold = 8.148381568597888

num_features = []
for col in initial_features:
    unique_values = X[col].nunique()
    
    if unique_values <= threshold:
        X[col] = X[col].astype('category').cat.as_ordered()
    else:
        num_features.append(col)

preprocessor = ColumnTransformer(
transformers=[
    ('num', StandardScaler(), num_features),
])

xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(**xgb_params)),
])

In [76]:
scores,oof = cross_validation(xgb, X, y, le, std_threshold)

Fold 1
Memory usage of dataframe is 18.28 MB
Memory usage after optimization is: 3.33 MB
Decreased by 81.8%


  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:10<00:00,  3.41it/s]


Fold 2
Memory usage of dataframe is 18.28 MB
Memory usage after optimization is: 3.33 MB
Decreased by 81.8%


100%|██████████| 36/36 [00:10<00:00,  3.45it/s]


...

Fold 4
Memory usage of dataframe is 18.28 MB
Memory usage after optimization is: 3.33 MB
Decreased by 81.8%


100%|██████████| 36/36 [00:10<00:00,  3.47it/s]


Fold 5
Memory usage of dataframe is 18.28 MB
Memory usage after optimization is: 3.33 MB
Decreased by 81.8%


100%|██████████| 36/36 [00:10<00:00,  3.52it/s]


Score: 0.8240344339946697


In [72]:
for fold in oof:
    fold['true'] = le.inverse_transform(fold['true'])

save_oof(oof)

In [28]:
X_test = test[initial_features]

predict_test(xgb, X, y, X_test, std_threshold, le.classes_)

Memory usage of dataframe is 22.23 MB
Memory usage after optimization is: 3.55 MB
Decreased by 84.0%


100%|██████████| 36/36 [00:13<00:00,  2.73it/s]


,pred_Dropout,pred_Enrolled,pred_Graduate
0,0.992321,0.004507,0.003172
1,0.009069,0.021329,0.969602
2,0.036337,0.271241,0.692423
3,0.264473,0.444803,0.290724
4,0.319181,0.631465,0.049355
...,...,...,...
51007,0.825931,0.067136,0.106933
51008,0.991588,0.007863,0.000548
51009,0.967134,0.017052,0.015814
51010,0.954300,0.037771,0.007928


In [73]:
oof[0]

,pred_Dropout,pred_Enrolled,pred_Graduate,true
0,0.052560,0.713844,0.233596,Graduate
1,0.916694,0.035887,0.047419,Dropout
2,0.967374,0.026351,0.006275,Dropout
3,0.891579,0.104180,0.004241,Dropout
4,0.978959,0.020275,0.000767,Dropout
...,...,...,...,...
16184,0.013626,0.017764,0.968610,Graduate
16185,0.068566,0.123258,0.808176,Graduate
16186,0.359713,0.512065,0.128223,Dropout
16187,0.027374,0.043467,0.929158,Graduate


In [74]:
scores = []
for fold in oof:
    model_cols = [col for col in fold.columns if col.startswith('pred')]
    y_true = fold['true']
    y_pred = fold[model_cols].idxmax(axis=1).apply(lambda x: x.split('_')[-1])

    score = accuracy_score(y_true, y_pred)
    print(score)

    scores.append(score)

0.825066403113225
0.8278460683179937
0.8240054361255251
0.8256733382752657
0.8175809241413393


In [77]:
scores

[0.825066403113225,
 0.8278460683179937,
 0.8240054361255251,
 0.8256733382752657,
 0.8175809241413393]

In [75]:
np.mean(scores)

0.8240344339946697

In [37]:
model_cols = [col for col in fold.columns if col.startswith('pred')]

In [38]:
model_cols

['pred_Dropout', 'pred_Enrolled', 'pred_Graduate']

In [63]:
fold

,pred_Dropout,pred_Enrolled,pred_Graduate,true
0,0.041509,0.502447,0.456044,Graduate
1,0.918601,0.035066,0.046334,Dropout
2,0.970203,0.024138,0.005658,Dropout
3,0.916458,0.080132,0.003411,Dropout
4,0.980102,0.019175,0.000723,Dropout
...,...,...,...,...
16184,0.014605,0.008770,0.976624,Graduate
16185,0.062404,0.057368,0.880228,Graduate
16186,0.423272,0.395291,0.181436,Dropout
16187,0.017502,0.029641,0.952857,Graduate


In [39]:
y_pred = fold[model_cols].idxmax(axis=1).apply(lambda x: x.split('_')[-1])

In [40]:
y_pred

0        Enrolled
1         Dropout
2         Dropout
3         Dropout
4         Dropout
           ...   
16184    Graduate
16185    Graduate
16186     Dropout
16187    Graduate
16188    Graduate
Length: 16189, dtype: object

In [67]:
le.inverse_transform(fold['true'])

ValueError: y contains previously unseen labels: ['Dropout' 'Enrolled' 'Graduate']

In [65]:
le.classes_

array(['Dropout', 'Enrolled', 'Graduate'], dtype=object)

In [61]:
fold['true'].unique()

array(['Graduate', 'Dropout', 'Enrolled'], dtype=object)